# Recreate IPCC AR6 ERF timeseries

- with updated data
- using AR6 assessments for components not directly observed

In [ ]:
import json

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import scipy.stats

In [ ]:
# probablistic ensemble
SAMPLES = 200
forcing = {}

In [ ]:
NINETY_TO_ONESIGMA = scipy.stats.norm.ppf(0.95)
NINETY_TO_ONESIGMA

In [ ]:
with open('../data/random_seeds.json', 'r') as filehandle:
    SEEDS = json.load(filehandle)

In [ ]:
emissions = pd.read_csv('../output/slcf_emissions_1750-2022.csv', index_col=0)
emissions

In [ ]:
concentrations = pd.read_csv('../output/ghg_concentrations_1750-2022.csv', index_col=0)
for year in range(1751, 1850):
    concentrations.loc[year, :] = np.nan
concentrations.sort_index(inplace=True)
concentrations.interpolate(inplace=True)

In [ ]:
concentrations

In [ ]:
# uncertainties from IPCC
uncertainty_seed = 38572

unc_ranges = np.array([
    0.12,      # CO2
    0.20,      # CH4: updated value from etminan 2016
    0.14,      # N2O
    0.19,      # other WMGHGs
    0.50,      # Total ozone
    1.00,      # stratospheric WV from CH4
    0.70,      # contrails approx - half-normal
    1.25,      # bc on snow - half-normal
    0.50,      # land use change
    5.0/20.0,  # volcanic
    0.50,      # solar (amplitude)
])/NINETY_TO_ONESIGMA

scale = scipy.stats.norm.rvs(
    size=(SAMPLES,11), 
    loc=np.ones((SAMPLES,11)), 
    scale=np.ones((SAMPLES, 11)) * unc_ranges[None,:], 
    random_state=uncertainty_seed
)

## BC snow is asymmetric Gaussian. We scale the half of the distribution above/below best estimate
scale[scale[:,7]<1,7] = 0.08/0.1*(scale[scale[:,7]<1,7]-1) + 1

## Contrails also asymmetric but benefits of scaling are tiny
scale[scale[:,6]<1,6] = 0.0384/0.0406*(scale[scale[:,6]<1,6]-1) + 1

trend_solar = scipy.stats.norm.rvs(
    size=SAMPLES, 
    loc=+0.01, 
    scale=0.07/NINETY_TO_ONESIGMA, 
    random_state=uncertainty_seed
)

scale_df = pd.DataFrame(
    data = scale,
    columns = ['co2','ch4','n2o','other_wmghg','o3','h2o_stratospheric','contrails','bc_on_snow','land_use','volcanic','solar']
)

In [ ]:
scale_df

In [ ]:
## put solar and volcanic here

## Aerosol forcing

In AR6, ERFari was based on emissions to forcing coefficients from Myhre et al (2013) https://acp.copernicus.org/articles/13/1853/2013/. At the time, I deemed there not sufficient evidence from CMIP6 AerChemMIP models or any other sources to update these. The uncertainty ranges from each precursor were expanded slightly compared to Myhre et al., in order to reproduce the overall ERFari uncertainty assessment (assumed that uncertainties in individual components are uncorrelated).

Following AR6 and a re-calibration of FaIR, 




ERFaci is based on fits to CMIP6 models from Smith et al. (2020) https://www.essoar.org/doi/abs/10.1002/essoar.10503977.2

Rescale both to the assessed forcings of -0.3 W/m2 for ERFari 2005-14 and -1.0 for ERFaci 2005-14.

In [ ]:
# these come from AR6 WG1
# they sum to -0.22 W/m2, for 2019
# Calculate a radiative efficiency for each species from CEDS and updated concentrations.

erfari_emitted = pd.Series(
    {
        "SO2": -0.234228,
        "BC": 0.144702,
        "OC": -0.072143,
        "NH3": -0.033769,
        "NOx": -0.009166,
        "NMVOC": -0.002573,
        "CO": 0,
        "CH4": -0.002653,
        "N2O": -0.00209,
        "EESC": -0.00808,
    }
)

In [ ]:
def calculate_eesc(
    concentration,
    fractional_release,
    fractional_release_cfc11,
    cl_atoms,
    br_atoms,
    br_cl_ratio=45,
):

    # EESC is in terms of CFC11-eq
    eesc_out = (
        cl_atoms * (concentration) * fractional_release / fractional_release_cfc11
        + br_cl_ratio
        * br_atoms
        * (concentration)
        * fractional_release
        / fractional_release_cfc11
    ) * fractional_release_cfc11
    return eesc_out


fractional_release = {
    "CFC-11": 0.47,
    "CFC-12": 0.23,
    "CFC-113": 0.29,
    "CFC-114": 0.12,
    "CFC-115": 0.04,
    "HCFC-22": 0.13,
    "HCFC-141b": 0.34,
    "HCFC-142b": 0.17,
    "CCl4": 0.56,
    "CHCl3": 0,
    "CH2Cl2": 0,
    "CH3Cl": 0.44,
    "CH3CCl3": 0.67,
    "CH3Br": 0.6,
    "Halon-1211": 0.62,
    "Halon-1301": 0.28,
    "Halon-2402": 0.65,
}

cl_atoms = {
    "CFC-11": 3,
    "CFC-12": 2,
    "CFC-113": 3,
    "CFC-114": 2,
    "CFC-115": 1,
    "HCFC-22": 1,
    "HCFC-141b": 2,
    "HCFC-142b": 1,
    "CCl4": 4,
    "CHCl3": 3,
    "CH2Cl2": 2,
    "CH3Cl": 1,
    "CH3CCl3": 3,
    "CH3Br": 0,
    "Halon-1211": 1,
    "Halon-1301": 0,
    "Halon-2402": 0,
}

br_atoms = {
    "CFC-11": 0,
    "CFC-12": 0,
    "CFC-113": 0,
    "CFC-114": 0,
    "CFC-115": 0,
    "HCFC-22": 0,
    "HCFC-141b": 0,
    "HCFC-142b": 0,
    "CCl4": 0,
    "CHCl3": 0,
    "CH2Cl2": 0,
    "CH3Cl": 0,
    "CH3CCl3": 0,
    "CH3Br": 1,
    "Halon-1211": 1,
    "Halon-1301": 1,
    "Halon-2402": 2,
}

hc_eesc = {}
total_eesc = np.zeros(273)
for species in cl_atoms:
    hc_eesc[species] = calculate_eesc(
        concentrations.loc[:, species],
        fractional_release[species],
        fractional_release["CFC-11"],
        cl_atoms[species],
        br_atoms[species],
    )
    total_eesc = total_eesc + hc_eesc[species]

total_eesc

In [ ]:
total_eesc = total_eesc.to_frame('EESC')

In [ ]:
# erfari radiative efficiency per Mt or ppb or ppt
re = erfari_emitted / (emissions.loc[2019, :] - emissions.loc[1750, :])
re.dropna(inplace=True)

In [ ]:
re['CH4'] = erfari_emitted['CH4'] / (concentrations.loc[2019, 'CH4'] - concentrations.loc[1750, 'CH4'])
re['N2O'] = erfari_emitted['N2O'] / (concentrations.loc[2019, 'N2O'] - concentrations.loc[1750, 'N2O'])
re['EESC'] = erfari_emitted['EESC'] / (total_eesc.loc[2019] - total_eesc.loc[1750])

In [ ]:
re

In [ ]:
erfari = pd.concat(
    (
        (re * emissions)[['BC', 'OC', 'SO2', 'NOx', 'CO', 'NMVOC', 'NH3']] - (re * emissions.loc[1750, ['BC', 'OC', 'SO2', 'NOx', 'CO', 'NMVOC', 'NH3']]),
        (re * concentrations)[['CH4', 'N2O']] - (re * concentrations.loc[1750, ['CH4', 'N2O']]),
        re['EESC'] * (total_eesc - total_eesc.loc[1750])
    ), axis=1
).dropna(axis=1)

In [ ]:
pl.plot(erfari.sum(axis=1))

In [ ]:
(erfari.sum(axis=1).loc[2005:2014]).mean()

In [ ]:
erfari.sum(axis=1)